In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd

In [2]:
# Set up data loading

#Data files
file_base = '../Data/'

data_types = ['filtered', 'raw']

celltype_strings = ['b_cells_','cd14_monocytes_','cd4_t_helper_','cd56_nk_','memory_t_','regulatory_t_']
celltype_strings = ['cd14_monocytes_','cytotoxic_t_','memory_t_','naive_cytotoxic_','naive_t_','regulatory_t_']
celltype_sub = '_gene_bc_matrices'

file_sub = '/matrices_mex/hg19/'

file_mtx = 'matrix.mtx'
file_bar = 'barcodes.tsv'
file_gen = 'genes.tsv'

In [3]:
# do for both datatypes
#data = []
for data_type in data_types[1:]:
    data_file = file_base+data_type+"/"+celltype_strings[0]+data_type+celltype_sub+file_sub

    #Load data
    adata = sc.read(data_file+file_mtx, cache=True)
    adata = adata.transpose()
    adata.X = adata.X.toarray()

    barcodes = pd.read_csv(data_file+file_bar, header=None, sep='\t')
    genes = pd.read_csv(data_file+file_gen, header=None, sep='\t')

    #Annotate data
    barcodes.rename(columns={0:'barcode'}, inplace=True)
    barcodes.set_index('barcode', inplace=True)
    adata.obs = barcodes

    adata.obs['celltype'] = [celltype_strings[0]]*adata.n_obs

    genes.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
    genes.set_index('gene_symbol', inplace=True)
    adata.var = genes
    
    print(adata)
    # Loop to load rest of data sets
    for celltype in celltype_strings[1:]:
        print(celltype)
        data_file = file_base+data_type+"/"+celltype+data_type+celltype_sub+file_sub

        #Load data
        adata_tmp = sc.read(data_file+file_mtx, cache=True)
        adata_tmp = adata_tmp.transpose()
        adata_tmp.X = adata_tmp.X.toarray()

        barcodes_tmp = pd.read_csv(data_file+file_bar, header=None, sep='\t')
        genes_tmp = pd.read_csv(data_file+file_gen, header=None, sep='\t')

        #Annotate data
        barcodes_tmp.rename(columns={0:'barcode'}, inplace=True)
        barcodes_tmp.set_index('barcode', inplace=True)
        adata_tmp.obs = barcodes_tmp
        adata_tmp.obs['celltype'] = [celltype]*adata_tmp.n_obs

        genes_tmp.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
        genes_tmp.set_index('gene_symbol', inplace=True)
        adata_tmp.var = genes_tmp
        adata_tmp.var_names_make_unique()
        
        print(adata_tmp)
        # Concatenate to main adata object
        adata = adata.concatenate(adata_tmp, batch_key='sample_id')
        adata.obs_names = [c.split("-")[0] for c in adata.obs_names]
        adata.obs_names_make_unique(join='_')
#    data.append(adata)

MemoryError: Unable to allocate array with shape (737280, 32738) and data type float32

In [10]:
import os
os.listdir('../Data/raw')

['b_cells_raw_gene_bc_matrices',
 'cd14_monocytes_raw_gene_bc_matrices',
 'cd34_raw_gene_bc_matrices',
 'cd4_t_helper_raw_gene_bc_matrices',
 'cd56_nk_raw_gene_bc_matrices',
 'cytotoxic_t_raw_gene_bc_matrices',
 'memory_t_raw_gene_bc_matrices',
 'naive_cytotoxic_raw_gene_bc_matrices',
 'naive_t_raw_gene_bc_matrices',
 'regulatory_t_raw_gene_bc_matrices']